In [2]:
import json
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from collections import OrderedDict 
import requests 

f = open('neighbor-districts-modified.json') 
neighbourData = json.load(f) 
f1 = open('data-all.json') 
dataAll = json.load(f1) 

In [3]:
df = pd.DataFrame(list((neighbourData.keys())),columns = [ 'districtname'])
district = []
for i in  range(len(df)): 
    district.append(df['districtname'][i][:(df['districtname'][i].index('/'))])
    
df.index = df.index + 101
df.insert(1, "districtid", df.index, True) 
df.insert(1, "districtnamecovid", district , True) 

In [4]:
districtId = pd.Series(df.districtid.values,index=df.districtnamecovid).to_dict()
districtname = pd.Series(df.districtname.values,index=df.districtid).to_dict()


In [5]:
covidDistrictId = {}
a = {}
for state in dataAll['2020-09-05']:
    covidDistrictId[state] = {}
    if "districts" in dataAll['2020-09-05'][state]:
        for district in dataAll['2020-09-05'][state]["districts"]:
            if district in districtId:
                covidDistrictId[state][district] = districtId[district]
                a[district+state] = districtId[district]

In [6]:
covidDistrictId

{'AN': {},
 'AP': {'Anantapur': 122,
  'Chittoor': 221,
  'East Godavari': 267,
  'Guntur': 307,
  'Krishna': 429,
  'Kurnool': 434,
  'Prakasam': 563,
  'Srikakulam': 665,
  'Visakhapatnam': 725,
  'Vizianagaram': 726,
  'West Godavari': 735},
 'AR': {'Anjaw': 124,
  'Changlang': 208,
  'East Kameng': 269,
  'East Siang': 271,
  'Kamle': 373,
  'Kra Daadi': 428,
  'Kurung Kumey': 436,
  'Lepa Rada': 446,
  'Lohit': 448,
  'Longding': 449,
  'Lower Dibang Valley': 451,
  'Lower Siang': 452,
  'Lower Subansiri': 453,
  'Namsai': 512,
  'Pakke Kessang': 537,
  'Papum Pare': 546,
  'Shi Yomi': 631,
  'Siang': 634,
  'Tawang': 679,
  'Tirap': 692,
  'Upper Dibang Valley': 708,
  'Upper Siang': 709,
  'Upper Subansiri': 710,
  'West Kameng': 737,
  'West Siang': 740},
 'AS': {},
 'BR': {'Araria': 126,
  'Arwal': 129,
  'Aurangabad': 132,
  'Banka': 149,
  'Begusarai': 163,
  'Bhagalpur': 170,
  'Bhojpur': 178,
  'Buxar': 196,
  'Darbhanga': 234,
  'Gaya': 293,
  'Gopalganj': 303,
  'Jamui':

In [7]:
covidDistrictId['UP']['Hamirpur'] = 312
covidDistrictId['BR']['Aurangabad'] = 131
covidDistrictId['CT']['Bilaspur'] = 184
covidDistrictId['UP']['Pratapgarh'] = 564
covidDistrictId['CT']['Balrampur'] = 145

In [8]:
dayCount = 0
week = 1
casesWeek = {}
casesMonth = {}
casesOverall = {}

for date in dataAll:
    
    day = int(date[8:11])
    month = int(date[5:7])
    overall = 1
    
    if (month == 3 and day >= 15) or (month >= 4 and month <=8) or (month == 9 and day <= 5):
        for state in dataAll[date]:
        
            if "districts" in dataAll[date][state]:
                for district in dataAll[date][state]["districts"]:
                    if district in districtId:
                        if "delta" in dataAll[date][state]["districts"][district]:
                            if "confirmed" in dataAll[date][state]["districts"][district]["delta"]:
                                if (covidDistrictId[state][district] in casesWeek) and (covidDistrictId[state][district] in casesMonth) and (covidDistrictId[state][district] in casesOverall):
                                    if (week in casesWeek[covidDistrictId[state][district]]):
                                        casesWeek[covidDistrictId[state][district]][week] = casesWeek[covidDistrictId[state][district]][week] + dataAll[date][state]["districts"][district]["delta"]['confirmed']
    
                                    else:
                                        casesWeek[covidDistrictId[state][district]][week] = dataAll[date][state]["districts"][district]["delta"]["confirmed"]
                                        
                                    if (month in casesMonth[covidDistrictId[state][district]]):
                                        casesMonth[covidDistrictId[state][district]][month-2] = casesMonth[covidDistrictId[state][district]][month] + dataAll[date][state]["districts"][district]["delta"]['confirmed']
                
                                    else:
                                        casesMonth[covidDistrictId[state][district]][month-2] = dataAll[date][state]["districts"][district]["delta"]["confirmed"]
                            
                                        
                                    if (overall in casesOverall[covidDistrictId[state][district]]):
                                        casesOverall[covidDistrictId[state][district]][overall] = casesOverall[covidDistrictId[state][district]][overall] + dataAll[date][state]["districts"][district]["delta"]["confirmed"]
                               
                                    else:
                                        casesOverall[covidDistrictId[state][district]][overall] = dataAll[date][state]["districts"][district]["delta"]["confirmed"]
                                        
                            
                                else:
                                    casesWeek[covidDistrictId[state][district]] = {}
                                    casesMonth[covidDistrictId[state][district]] = {}
                                    casesOverall[covidDistrictId[state][district]] = {}
                                    casesWeek[covidDistrictId[state][district]][week] = dataAll[date][state]["districts"][district]["delta"]["confirmed"]
                                    casesMonth[covidDistrictId[state][district]][month-2] = dataAll[date][state]["districts"][district]["delta"]["confirmed"]
                                    casesOverall[covidDistrictId[state][district]][overall] = dataAll[date][state]["districts"][district]["delta"]["confirmed"]
                               
                      
        dayCount = dayCount + 1  
        if dayCount == 7:
            dayCount = 0
            week = week + 1

In [9]:
rows = [] 

for data in casesWeek: 
    data_row = casesWeek[data]
    districtId = data
    row = {}
    for time in data_row: 
        rows.append({'districtid':districtId, 'timeid': time, 'cases':data_row[time] }) 

dfWeek = pd.DataFrame(rows) 

In [10]:
rows = [] 

for data in casesMonth: 
    data_row = casesMonth[data]
    districtId = data
    row = {}
    for time in data_row: 
        rows.append({'districtid':districtId, 'timeid': time, 'cases':data_row[time] }) 

dfMonth = pd.DataFrame(rows) 

In [11]:
rows = [] 

for data in casesOverall: 
    data_row = casesOverall[data]
    districtId = data
    row = {}
    for time in data_row: 
        rows.append({'districtid':districtId, 'timeid': time, 'cases':data_row[time] }) 

dfOverall = pd.DataFrame(rows) 

In [12]:
dfWeek.to_csv (r'cases-week.csv', index = False, header=True)

In [13]:
dfMonth.to_csv (r'cases-month.csv', index = False, header=True)

In [14]:
dfOverall.to_csv (r'cases-overall.csv', index = False, header=True)

In [17]:
districtname

{101: 'Adilabad/Q15211',
 102: 'Agar Malwa/Q15732396',
 103: 'Agra/Q606343',
 104: 'Ahmedabad/Q401686',
 105: 'Ahmednagar/Q401744',
 106: 'Ajmer/Q413037',
 107: 'Akola/Q520510',
 108: 'Alappuzha/Q928959',
 109: 'Aligarh/Q766918',
 110: 'Alipurduar/Q4726845',
 111: 'Alirajpur/Q2667586',
 112: 'Almora/Q1805066',
 113: 'Alwar/Q449690',
 114: 'Ambala/Q2086226',
 115: 'Ambedkar Nagar/Q456764',
 116: 'Amethi/Q1071494',
 117: 'Amravati/Q1771774',
 118: 'Amreli/Q257946',
 119: 'Amritsar/Q202822',
 120: 'Amroha/Q1891677',
 121: 'Anand/Q485683',
 122: 'Anantapur/Q15212',
 123: 'Anantnag/Q2982349',
 124: 'Anjaw/Q15413',
 125: 'Anuppur/Q2299093',
 126: 'Araria/Q42901',
 127: 'Aravalli/Q12175285',
 128: 'Ariyalur/Q15112',
 129: 'Arwal/Q42917',
 130: 'Auraiya/Q1812533',
 131: 'Aurangabad/Q43086',
 132: 'Aurangabad/Q592942',
 133: 'Ayodhya/Q1814132',
 134: 'Azamgarh/Q793553',
 135: 'Bagalkote/Q1910231',
 136: 'Bageshwar/Q1815313',
 137: 'Baghpat/Q1797363',
 138: 'Bahraich/Q1812548',
 139: 'Baksa/Q236